## Setup

In [ ]:
# import packages and modules
import os,arcpy
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor
import pandas as pd
from functools import reduce
import pyodbc

# set data frame display options
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# set overwrite to true
arcpy.env.overwriteOutput = True

# in memory output file path
wk_memory = "memory" + "\\"

# set workspace and sde connections 
scratchFolder = "C:\\GIS"
workspace     = "//Trpa-fs01/GIS/PROJECTS/ResearchAnalysis/MailingLists"
desktop       = "C:\\Users\\mbindl\\Desktop"
arcpy.env.workspace = "C:\\GIS\\Scratch.gdb"

## SDE Connection Files saved on the Network
# sdeTabular = "F:\\GIS\\GIS_DATA\\Tabular.sde"
# sdeBase    = "F:\\GIS\\GIS_DATA\\Vector.sde"
# sdeCollect = "F:\\GIS\\GIS_DATA\\Collect.sde"

# network path to connection files
filePath = "C:\\GIS\\DB_CONNECT"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")

### IPES Mailing List
*	Douglas County, Washoe County, El Dorado County, and City of South Lake Tahoe parcels with an IPES score of 0
*	Placer County parcels with an IPES score below 726
*	Only include vacant lots
*	Exclude properties under public ownership (USFS, CTC, State, Local Jurisdiction, etc.)
*	Exclude properties deed restricted as open space and/or previously retired 


In [ ]:
# get IPES data from LTinfo
dfIPES     = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

In [ ]:
# get LTinfo data
dfIPES     = pd.read_json("https://laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfDevRight = pd.read_json('https://laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfDeed     = pd.read_json('https://laketahoeinfo.org/WebServices/GetDeedRestrictedParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')

# parcel master
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"

# create spatial dataframe from parcel feature class
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

In [ ]:
# sql database connection to BMP with pyodbc
bmpConnect = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql14;DATABASE=tahoebmpsde;UID=sde;PWD=staff')
# BMP - create dataframes from tahoebmpsde
dfParcels     = pd.read_sql("SELECT * FROM [tahoebmpsde].[sde].[TblParcel]", bmpConnect)
dfParcels.rename(columns={"APN_String": "APNs"}, inplace=True)
dfParcels

In [ ]:
dfDevRight.RecordType.unique()

In [ ]:
#filter for Official Active IPES
dfIPESactive = dfIPES[(dfIPES.Status == 'Active')&(dfIPES.IPESScoreType == 'Official')]

# filter to Retired
dfRetired = dfDeed[(dfDeed.DeedRestrictionType == "Retired")]
# drop duplicates
dfRetired = dfRetired.drop_duplicates('APN')

# filter Allocations from all Dev Rights
dfAll = dfDevRight[(dfDevRight.RecordType == 'Allocation Assignment Receiving Parcel') | 
                   (dfDevRight.RecordType == 'Allocation Receiving Parcel') |
                   (dfDevRight.RecordType == 'Allocation Assignment Sending Parcel')]

dfAllocation = dfAll.drop_duplicates('APN')

In [ ]:
data_frames = [dfIPESactive, dfRetired, dfAllocation, sdfParcels]

# merge dataframes
df = reduce(lambda  left,right: pd.merge(left,right,on=['APN'], how='left'), data_frames)
df.shape

In [ ]:
df.rename(columns={'IPESScore_x': 'IPESScore'}, inplace=True)
# specify fields to keep
dfOut = df[['APN',
            'PPNO',
            'Status',
            'IPESScore',
            'IPESScoreType',
            'DeedRestrictionType',
            'RecordType',
            'DevelopmentRight',
            'APO_ADDRESS',
            'PSTL_TOWN',
            'PSTL_STATE',
            'PSTL_ZIP5',
            'OWN_FIRST',
            'OWN_LAST',
            'OWN_FULL',
            'MAIL_ADD1',
            'MAIL_ADD2',
            'MAIL_CITY',
            'MAIL_STATE',
            'MAIL_ZIP5',
            'JURISDICTION',
            'COUNTY',
            'OWNERSHIP_TYPE',
            'COUNTY_LANDUSE_DESCRIPTION',
            'EXISTING_LANDUSE',
            'REGIONAL_LANDUSE',
            'PARCEL_ACRES'
            ]].copy()

df = dfOut

In [ ]:
# filter by all criteria
df = df[(((df.COUNTY == 'PL') & (df.IPESScore < 726)) | ((df.COUNTY.isin(['EL','WA','DG','CC'])) & (df.IPESScore == 0)))
        & (df.DeedRestrictionType.isnull()) & (df.DevelopmentRight.isnull())
        & ((df.OWNERSHIP_TYPE == 'Private') & (df.EXISTING_LANDUSE == 'Vacant'))]

df.shape

In [ ]:
dfParcels.drop(columns=['APN'],inplace=True)

In [ ]:
# dfParcels.drop(columns=['APN'],inplace=True)
dfParcels.rename(columns={'APNs': 'APN'}, inplace=True)
# dfParcels.rename(columns={'APN': 'PPNO'}, inplace=True)
dfNewAddress = pd.merge(df, dfParcels, on=['APN'], how='left')


dfMail = dfNewAddress[['APN',
            'Status',
            'IPESScore',
            'IPESScoreType',
            'DeedRestrictionType',
            'RecordType',
            'DevelopmentRight',
            'APO_ADDRESS',
            'PSTL_TOWN',
            'PSTL_STATE',
            'PSTL_ZIP5',
            'OWN_FIRST',
            'OWN_LAST',
            'OWN_FULL',
            'MAIL_ADD1',
            'MAIL_ADD2',
            'MAIL_CITY',
            'MAIL_STATE',
            'MAIL_ZIP5',
            'OwnerName',
            'OwnerFirst',
            'OwnerLast',
            'OwnerStreet',
            'OwnerCity',
            'OwnerState',
            'OwnerZip',
            'OwnerCountry',
            'OwnerPhone',
            'OwnerEmail',       
            'JURISDICTION',
            'COUNTY',
            'OWNERSHIP_TYPE',
            'COUNTY_LANDUSE_DESCRIPTION',
            'EXISTING_LANDUSE',
            'REGIONAL_LANDUSE',
            'PARCEL_ACRES'
            ]].copy()

dfMail.to_csv(os.path.join(workspace, 'IPES_MailingList.csv'))

### Deed Restriction Mailing List

In [ ]:
# get Deed Restriction data from lTinfo
dfDeed     = pd.read_json("https://laketahoeinfo.org/WebServices/GetDeedRestrictedParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

In [ ]:
# copy parcel master to local gdb
# parcel base
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
# # copy
# arcpy.FeatureClassToFeatureClass_conversion(parcels, workspace,"ParcelMaster")

# create spatial dataframe from parcel feature class
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

sdfParcels.info()

In [ ]:
# merge parcels and sql table on APN
df = pd.merge(sdfParcels, dfDeed, on='APN', how='right')



# specify fields to keep
## Add Physical Address Information and Registration Status
dfOut = df[['APN',
            'RecordingNumber',
            'RecordingDate',
            'Description',
            'DeedRestrictionStatus',
            'DeedRestrictionType',
            'ProjectAreaFileNumber',
            'OWN_FULL',
            'APO_ADDRESS',
            'PSTL_TOWN',
            'PSTL_STATE',
            'PSTL_ZIP5',            
            'MAIL_ADD1',
            'MAIL_ADD2',
            'MAIL_CITY',
            'MAIL_STATE',
            'MAIL_ZIP5',
            'COUNTY'
            ]].copy()

dfOut

# dfM2= dfMoor.rename(columns={"Registration Submission #": "Registration"})

# dfFinal = pd.merge(dfOut, dfM2, on='Registration', how='left')

In [ ]:
dfOut.to_csv(os.path.join(desktop,'DeedRestriction_MailMerge.csv'))

### Mooring Registration Mailing List

In [1]:
##--------------------------------------------------------------------------------------##
## SETUP
# import packages
import os
import pandas as pd
# external connection packages
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

# workspace to save csvs
workspace = r"F:\GIS\PROJECTS\ResearchAnalysis\MailingLists\Mooring"

# setup SQL connection engine
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql14;DATABASE=tahoebmpsde;UID=sde;PWD=staff"
connection_url    = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine            = create_engine(connection_url)

##--------------------------------------------------------------------------------------##
## EXTRACT

# use the sql engine
with engine.begin() as bmpConnect:
    # Create parcel dataframes from tahoebmpsde
    dfParcels = pd.read_sql("SELECT * FROM [tahoebmpsde].[sde].[TblParcel]", bmpConnect)

# table downloaded from parcel tracker 
# REGISTRATIONS SUMBISSIONS REVIEW grid with 'All Registration Submissions' checked
# https://parcels.laketahoeinfo.org/MooringRegistrationSubmissionReview/AllMooringRegistrationSubmissions
dfMoor = pd.read_csv(os.path.join(workspace, "mooringRegistrationsGrid.csv"))

##---------------------------------------------------------------------------------------##
## TRANSFORM

# Stripping out the first character
dfMoor['Registration'] = dfMoor['Registration Submission #'].str[1:]
# fill NaN with 0 and convert to integer
dfMoor['Registration'] = dfMoor['Registration'].fillna(0).astype(int)
# Changing the data type 
dfMoor['Registration'] = dfMoor['Registration'].astype(int)
# Changing the data type 
dfMoor['APNs'] = dfMoor['APNs'].astype(str)
# create series of column values listing APNs indexed by registration number
stackedMoor = pd.DataFrame(dfMoor.APNs.str.split(',').tolist(), index=dfMoor['Registration']).stack()
# convert series to a multi-indexed data frame
df = stackedMoor.to_frame()
# format to dataframe
flatMoor = pd.DataFrame(df.to_records())
# rename the index column
df = flatMoor.rename(columns={"0": "APN"})
# strip off blank spaces around the stacked APN values
df.APN = df.APN.str.strip()
# drop leftover field from flattening
df.drop('level_1', axis=1, inplace=True)

# merge orginal df to get list of APNs in stacked df
df = pd.merge(df, dfMoor[['Registration', 'APNs','Registration Status']], how='left', on='Registration')
# merge parcels to df
df = pd.merge(df, dfParcels, how='left', left_on='APN', right_on='APN_String')

# rename the joined APN field
df.rename(columns={"APN_x": "APN"}, inplace=True)

# specify fields to keep
dfOut = df[['Registration',
            'Registration Status',
            'APN',
            'APNs',
            'ParcelStreet',
            'ParcelCity',
            'ParcelZip',
            'OwnerName',
            'OwnerStreet',
            'OwnerCity',
            'OwnerState',
            'OwnerZip',
            'OwnerPhone',
            'OwnerEmail'      
            ]].copy()

# not sure if we should drop dupes and NaN records?
# dfOut.drop_duplicates(subset="APN", inplace=True)

##--------------------------------------------------------------------------------------##
## LOAD

dfOut.to_csv(os.path.join(workspace, "MooringRegistration_MailingList.csv"))

### Mailing List from Buffer

In [ ]:

# Set the input workspace, get the feature class name to copy
#  and the output location.
arcpy.env.workspace = arcpy.GetParameterAsText(0)
in_featureclass = arcpy.GetParameterAsText(1)
out_workspace = arcpy.GetParameterAsText(2)

out_featureclass = os.path.join(out_workspace,
                                os.path.basename(in_featureclass))


In [ ]:
# copy parcel master to local gdb
# parcel base
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
# # copy
# arcpy.FeatureClassToFeatureClass_conversion(parcels, workspace,"ParcelMaster")

# create spatial dataframe from parcel feature class
sdfParcels = pd.DataFrame.spatial.from_featureclass(parcels)

sdfParcels.info()

In [ ]:
arcpy.management.SelectLayerByAttribute("Parcels", "NEW_SELECTION", "APN IN ("+ apns + ")", None)

arcpy.management.SelectLayerByLocation(
    "Parcels", 
    "HAVE_THEIR_CENTER_IN", 
    "Parcels", 
    "300 Feet", 
    "ADD_TO_SELECTION", 
    "NOT_INVERT")

arcpy.conversion.TableToTable("Parcels", 
                              r"C:\GIS", 
                              "Parcel_MailingList.csv", 
                              '', 
                              'APN "APN" true true false 16 Text 0 0,First,#,Parcels,APN,0,16;APO_ADDRESS "Full Address" true true false 100 Text 0 0,First,#,Parcels,APO_ADDRESS,0,100;PSTL_TOWN "Postal Town" true true false 25 Text 0 0,First,#,Parcels,PSTL_TOWN,0,25;PSTL_STATE "Postal State" true true false 2 Text 0 0,First,#,Parcels,PSTL_STATE,0,2;PSTL_ZIP5 "Postal Zip Code" true true false 5 Text 0 0,First,#,Parcels,PSTL_ZIP5,0,5;OWN_FIRST "Own First Name" true true false 50 Text 0 0,First,#,Parcels,OWN_FIRST,0,50;OWN_LAST "Owner Last Name" true true false 100 Text 0 0,First,#,Parcels,OWN_LAST,0,100;OWN_FULL "Owner Full Name" true true false 100 Text 0 0,First,#,Parcels,OWN_FULL,0,100;MAIL_ADD1 "Mailing Address 1" true true false 100 Text 0 0,First,#,Parcels,MAIL_ADD1,0,100;MAIL_ADD2 "Mailing Address 2" true true false 100 Text 0 0,First,#,Parcels,MAIL_ADD2,0,100;MAIL_CITY "Mailing City" true true false 50 Text 0 0,First,#,Parcels,MAIL_CITY,0,50;MAIL_STATE "Mailing State" true true false 2 Text 0 0,First,#,Parcels,MAIL_STATE,0,2;MAIL_ZIP5 "Mailing Zip Code" true true false 5 Text 0 0,First,#,Parcels,MAIL_ZIP5,0,5;JURISDICTION "Jurisdiction" true true false 4 Text 0 0,First,#,Parcels,JURISDICTION,0,4;OWNERSHIP_TYPE "Ownership Type" true true false 12 Text 0 0,First,#,Parcels,OWNERSHIP_TYPE,0,12;EXISTING_LANDUSE "Existing Land Use" true true false 50 Text 0 0,First,#,Parcels,EXISTING_LANDUSE,0,50;GlobalID "GlobalID" false false true 38 GlobalID 0 0,First,#,Parcels,GlobalID,-1,-1', 
                              '')

In [ ]:
# create series of column values listing APNs indexed by registration number
records = pd.DataFrame(dfMoor.APNs.str.split(',').tolist(), index=dfMoor['Registration Submission #']).stack()

# convert series to data frame
df = records.to_frame()

# format data frame
flattened = pd.DataFrame(df.to_records())
dfM = flattened.rename(columns={"Registration Submission #": "Registration", "0": "APN"})
dfM.drop(['level_0'], axis=1, inplace=True)
dfM
dfM.APN = dfM.APN.str.strip()

# merge parcels and sql table on APN
df = pd.merge(sdfParcels, dfM, on='APN', how='right')



# specify fields to keep
## Add Physical Address Information and Registration Status
dfOut = df[['APN',
            'Registration',
            'OWN_FULL',
            'APO_ADDRESS',
            'PSTL_TOWN',
            'PSTL_STATE',
            'PSTL_ZIP5',            
            'MAIL_ADD1',
            'MAIL_ADD2',
            'MAIL_CITY',
            'MAIL_STATE',
            'MAIL_ZIP5',
            'COUNTY'
            ]].copy()

dfOut

dfM2= dfMoor.rename(columns={"Registration Submission #": "Registration"})

dfFinal = pd.merge(dfOut, dfM2, on='Registration', how='left')

## Send Email

In [ ]:
import getpass, smtplib, os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# fileToSend = input("File: ")
# subject = input("Subject: ")
# sender_email = input("Outlook Email: ")
# password = getpass.getpass("Password: ")
# receiver_email = input("Recieving Email: ")

fileToSend = r"C:\Users\mbindl\Desktop\DeedRestriction_MailMerge.csv"
# subject = "Deed Restriction Mailing List"
# sender_email = "mbindl@trpa.gov"
# password = "Ma$eWork23"
# receiver_email = "mbindl@trpa.gov"

subject = "Deed Restriction Mailing List"
sender_email = "infosys@trpa.org"
# password = ''
receiver_email = "gis@trpa.gov"

def send_test_mail(body):
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = receiver_email

    msgText = MIMEText('%s<br><br>Cheers,<br>GIS Team' % (body), 'html')
    msg.attach(msgText)

    attachment = MIMEText(open(fileToSend).read())
    attachment.add_header("Content-Disposition", "attachment", filename = os.path.basename(fileToSend))
    msg.attach(attachment)

    try:
        with smtplib.SMTP('smtp.office365.com', 587) as smtpObj:
            smtpObj.ehlo()
            smtpObj.starttls()
            smtpObj.login(sender_email, password)
            smtpObj.sendmail(sender_email, receiver_email, msg.as_string())
    except Exception as e:
        print(e)

send_test_mail("Here's that list.")
print('Sending email...')